## Simulación _tick-by-tick_

Es uno de los modos de simulación más sencillos. En este caso, el tiempo total de simulación es dividido en pequeños intervalos. En cada intervalo (_tick_), el programa verifica todas las actividades involucradas en el sistema modelado.

El algoritmo general de este tipo de simulación es:

    MIENTRAS el tiempo simulación no termine
        aumentar tiempo en una unidad
        si ocurren eventos en este intervalo de tiempo:
            simular los eventos
            

Por ejemplo, consideremos el caso de modelar un taller de revisión técnica. En una revisión técnica, llegan vehículos (o clientes) cada cierto tiempo aleatorio, los que se colocan en una cola de espera. Además, atender cada vehículo toma otro tiempo aleatorio. Veamos como modelar esto con una simulación _tick-by-tick_.

In [1]:
from collections import deque
from random import choice, randrange, random, expovariate
from statistics import mean


class Vehículo:
    """ Esta clase modela los vehículos que llegan al taller."""

    def __init__(self, vehículos):
        # Cuando se crea un nuevo vehículo se escoge uniformemente el tipo de vehículo
        self.tipo = choice(list(vehículos))
        # Seleccionamos el tiempo que demorará atender a este vehículo.
        # Esperamos que como mínimo sea 1 minuto.
        self.tiempo_revisión = max(1, round(expovariate(vehículos[self.tipo])))

    def __repr__(self):
        return self.tipo


class Taller:
    """ Esta clase modela la línea de revisión en el taller."""

    def __init__(self):
        self.tarea_actual = None
        self.tiempo_revisión_restante = 0

    @property
    def ocupado(self):
        return self.tarea_actual is not None

    def atender(self, vehículo):
        self.tarea_actual = vehículo
        self.tiempo_revisión_restante = vehículo.tiempo_revisión
        print(f'[PLANTA] Atendiendo {self.tarea_actual}.',
              f'demorará {self.tiempo_revisión_restante} min')

    def tick(self):
        if self.tarea_actual is not None:
            self.tiempo_revisión_restante -= 1
            if self.tiempo_revisión_restante <= 0:
                print(f'[PLANTA] termina revisión de {self.tarea_actual}')
                self.tarea_actual = None


class Simulación:

    STR_TEMPLATE = ('[COLA] Llega {} en tiempo de simulación t={} min. '
                    'Hay {} vehículos en cola')

    def __init__(self, max_tiempo, vehículos):
        self.max_tiempo = max_tiempo
        self.vehículos = vehículos

        self.planta = Taller()
        self.cola_revisión = deque()
        self.tiempos_revisión = []

    @property
    def llega_nuevo_auto(self):
        """
        Esta property modela si llega o no un auto nuevo a la cola. 
        Se muestrea de una distribución de probabilidad uniforme. El método retorna
        True si el valor entregado por la función random es mayor a un valor dado.
        En este caso, es sencillo ver que retornará True un 20% de las veces.
        """
        return random() <= 0.2

    def imprimir_estadísticas(self):
        tiempo_promedio = mean(self.tiempos_revisión)
        tiempo_total = sum(self.tiempos_revisión)

        print()
        print('Estadísticas:')
        print(f'Tiempo promedio de atención {tiempo_promedio:6.2f} min.')
        print(f'Tiempo total de atención {tiempo_total:6.2f} min')
        print(f'Total de vehículos atendidos: {len(self.tiempos_revisión)}')

    def revisión_técnica(self):
        """Esta función maneja el proceso o servicio de revisión en el taller."""

        # Hacemos avanzar el reloj de la simulación minuto por minuto.
        # No importa si en ese minuto no pasa nada.
        for minuto in range(self.max_tiempo):

            if self.llega_nuevo_auto:
                vehículo = Vehículo(self.vehículos)
                self.cola_revisión.append(vehículo)
                print(
                    self.STR_TEMPLATE.format(vehículo, minuto,
                                             len(self.cola_revisión)))

            if self.cola_revisión and not self.planta.ocupado:
                # Si hay vehículos esperando y la planta no está ocupada
                # Atendemos un vehículo, sacándolo de la cola
                vehículo = self.cola_revisión.popleft()
                self.tiempos_revisión.append(vehículo.tiempo_revisión)
                self.planta.atender(vehículo)

            # Avisamos a la planta que ha transcurrido 1 minuto
            self.planta.tick()

        # Ya terminamos la simulación, imprimamos las estadísticas
        self.imprimir_estadísticas()


if __name__ == '__main__':
    # Define los tipos de vehículos y su tiempo de atención promedio
    vehículos = {'moto': 1 / 8, 'auto': 1 / 15, 'camioneta': 1 / 20}
    max_tiempo = 80

    simulación = Simulación(max_tiempo, vehículos)
    simulación.revisión_técnica()

[COLA] Llega auto en tiempo de simulación t=3 min. Hay 1 vehículos en cola
[PLANTA] Atendiendo auto. demorará 4 min
[PLANTA] termina revisión de auto
[COLA] Llega moto en tiempo de simulación t=16 min. Hay 1 vehículos en cola
[PLANTA] Atendiendo moto. demorará 19 min
[COLA] Llega auto en tiempo de simulación t=22 min. Hay 1 vehículos en cola
[COLA] Llega auto en tiempo de simulación t=33 min. Hay 2 vehículos en cola
[PLANTA] termina revisión de moto
[PLANTA] Atendiendo auto. demorará 21 min
[COLA] Llega camioneta en tiempo de simulación t=39 min. Hay 2 vehículos en cola
[COLA] Llega moto en tiempo de simulación t=42 min. Hay 3 vehículos en cola
[COLA] Llega moto en tiempo de simulación t=44 min. Hay 4 vehículos en cola
[COLA] Llega camioneta en tiempo de simulación t=52 min. Hay 5 vehículos en cola
[PLANTA] termina revisión de auto
[COLA] Llega moto en tiempo de simulación t=56 min. Hay 6 vehículos en cola
[PLANTA] Atendiendo auto. demorará 11 min
[PLANTA] termina revisión de auto
[PLA